In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/prop-3.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
0      prop-3      225    7    5    0   21   23     9   0  21  ...  0.000000   
1      prop-3      225    4    3    0    5   16     4   0   5  ...  0.333333   
2      prop-3      225    3    3    0   21   28     3   0  21  ...  0.000000   
3      prop-3      225    5    5    0   23   28     4   0  23  ...  0.000000   
4      prop-3      225    5    4    0   14   21     4   1  13  ...  0.000000   
...       ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
10269  prop-3      265    2    3    0   10    8     1   1   9  ...  0.000000   
10270  prop-3      265    7    2    0   29   19    21  22   7  ...  0.000000   
10271  prop-3      265    2    3    0   10    8     1   1   9  ...  0.000000   
10272  prop-3      265    3    3    0   29   42     3   1  28  ...  0.000000   
10273  prop-3      265    2    1    0   78   80     1   0  78  ...  1.000000   

       moa       mfa       cam  ic  cbm         amc  max_cc  avg_cc  bug  
0        0  0.827586  0.466667   3    4   11.000000       1  0.7143    0  
1        0  0.909091  0.666667   1    2   14.750000       2  0.7500    0  
2        0  0.750000  0.444444   1    1   43.666667       7  3.3333    0  
3        0  0.888889  0.666667   2    3   23.800000       1  0.6000    0  
4        0  0.888889  0.583333   3    5   17.600000       1  0.6000    0  
...    ...       ...       ...  ..  ...         ...     ...     ...  ...  
10269    0  0.857143  0.625000   1    1   11.500000       2  1.0000    0  
10270    0  0.400000  0.500000   1    1   28.857143      12  2.4286    0  
10271    0  0.857143  0.666667   1    1   10.500000       1  0.5000    0  
10272    0  0.750000  0.416667   1    1   60.000000      12  4.6667    0  
10273    0  0.000000  1.000000   0    0  160.000000       0  0.0000    0  

[10274 rows x 23 columns]

In [4]:
ant_master = ant_master.loc[ant_master['version'] == 225]
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
0     prop-3      225    7    5    0   21   23     9   0  21  ...  0.000000   
1     prop-3      225    4    3    0    5   16     4   0   5  ...  0.333333   
2     prop-3      225    3    3    0   21   28     3   0  21  ...  0.000000   
3     prop-3      225    5    5    0   23   28     4   0  23  ...  0.000000   
4     prop-3      225    5    4    0   14   21     4   1  13  ...  0.000000   
...      ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
1805  prop-3      225    1    1    0    1    5     0   0   1  ...  0.000000   
1806  prop-3      225    3    1    0    9   30     3   0   9  ...  0.000000   
1807  prop-3      225    1    1    0    0    5     0   0   0  ...  0.000000   
1808  prop-3      225    1    1    0    0    5     0   0   0  ...  0.000000   
1809  prop-3      225    2    4    0   17   15     1   0  17  ...  0.000000   

      moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0       0  0.827586  0.466667   3    4  11.000000       1  0.7143    0  
1       0  0.909091  0.666667   1    2  14.750000       2  0.7500    0  
2       0  0.750000  0.444444   1    1  43.666667       7  3.3333    0  
3       0  0.888889  0.666667   2    3  23.800000       1  0.6000    0  
4       0  0.888889  0.583333   3    5  17.600000       1  0.6000    0  
...   ...       ...       ...  ..  ...        ...     ...     ...  ...  
1805    0  0.000000  0.000000   0    0  12.000000       0  0.0000    0  
1806    0  0.000000  0.466667   0    0  65.333333       8  4.0000    0  
1807    0  0.000000  0.000000   0    0  12.000000       0  0.0000    0  
1808    0  0.000000  0.000000   0    0  12.000000       0  0.0000    0  
1809    0  0.888889  0.666667   2    2  34.500000       1  0.5000    0  

[1810 rows x 23 columns]

In [5]:
train, test = train_test_split(ant_master, test_size=0.2)

In [6]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [7]:
x_train

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc  dam  moa  \
958     2    3    0   15   12     1   0  15    2  2.000000   62  0.0    0   
300     3    1    0   10    3     3   4   6    3  2.000000    3  0.0    0   
0       7    5    0   21   23     9   0  21    2  0.888889   87  0.0    0   
107     6    3    0   13   19    11   0  13    4  0.950000   96  0.0    0   
1109    8    5    0   25   26    16   0  25    3  0.904762  116  0.0    0   
...   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...  ...  ...   
562     8    1    0   31   26    24  21  10    5  0.928571  102  0.5    0   
619     2    3    0   11   14     1   0  11    2  2.000000   45  0.0    0   
1475   11    4    0   36   66    33   7  29    8  0.900000  376  0.0    0   
853     5    5    0   17   18     4   0  17    1  0.916667   67  0.0    0   
267    11    4    0   37   69    41   5  32    9  0.925000  522  0.0    0   

           mfa       cam  ic  cbm        amc  max_cc  avg_cc  
958   0.857143  0.600000   1    1  30.000000       3  1.5000  
300   0.000000  0.481481   0    0   0.000000       1  1.0000  
0     0.827586  0.466667   3    4  11.000000       1  0.7143  
107   0.833333  0.533333   2    2  14.333333       2  0.8333  
1109  0.800000  0.666667   3    3  13.125000       1  0.7500  
...        ...       ...  ..  ...        ...     ...     ...  
562   0.000000  0.321429   0    0  11.250000       2  1.1250  
619   0.857143  0.625000   1    1  21.500000       3  1.5000  
1475  0.790698  0.766667   3    8  32.818182       2  0.9091  
853   0.888889  0.666667   2    3  11.800000       1  0.6000  
267   0.790698  0.766667   3    8  46.090909       2  0.9091  

[1448 rows x 20 columns]

In [8]:
x_test

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc       dam  \
1528    2    1    0    9   12     1   0   9    2  2.000000   50  1.000000   
1712    2    1    0    9   15     1   1   8    1  2.000000   52  0.000000   
1196    9    1    0   41   70    30   9  32    6  0.916667  621  0.333333   
1715    2    5    0    9    7     1   0   9    2  2.000000   31  0.000000   
565     7    6    0   18   36    15   0  18    2  0.944444  132  0.000000   
...   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...       ...   
1014    2    3    0   11   12     1   0  11    2  2.000000   64  0.000000   
787     6    5    0   13   18     9   0  13    2  0.933333   71  0.000000   
305     4    3    0   16   38     2   0  16    1  0.933333  361  0.200000   
1157   11    5    0   50   91    23   5  45    9  0.875000  576  0.000000   
1175    6    5    0   17   35    15   0  17    2  1.066667  179  0.000000   

      moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  
1528    0  0.000000  0.666667   0    0  23.500000       4  2.0000  
1712    0  0.000000  0.500000   0    0  25.000000       1  0.5000  
1196    1  0.000000  0.225000   0    0  67.666667      30  5.2222  
1715    0  0.923077  0.666667   0    0  14.500000       4  2.0000  
565     0  0.848485  0.555556   3    7  17.428571       1  0.7143  
...   ...       ...       ...  ..  ...        ...     ...     ...  
1014    0  0.857143  0.600000   1    1  31.000000       5  2.5000  
787     0  0.857143  0.600000   3    3  10.333333       1  0.6667  
305     0  0.909091  0.666667   2    3  88.000000      16  5.5000  
1157    0  0.800000  0.766667   4   10  51.000000       2  0.9091  
1175    0  0.857143  0.333333   3    3  28.333333       3  1.0000  

[362 rows x 20 columns]

In [9]:
y_train = train['bug']
y_test = test['bug']

In [10]:
y_train

958     0
300     0
0       0
107     0
1109    0
       ..
562     0
619     0
1475    0
853     0
267     0
Name: bug, Length: 1448, dtype: int64

In [11]:
y_test

1528    0
1712    0
1196    1
1715    0
565     0
       ..
1014    0
787     0
305     0
1157    0
1175    0
Name: bug, Length: 362, dtype: int64

In [12]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [13]:
x_train_np

array([[ 2.        ,  3.        ,  0.        , ..., 30.        ,
         3.        ,  1.5       ],
       [ 3.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ],
       [ 7.        ,  5.        ,  0.        , ..., 11.        ,
         1.        ,  0.7143    ],
       ...,
       [11.        ,  4.        ,  0.        , ..., 32.81818182,
         2.        ,  0.9091    ],
       [ 5.        ,  5.        ,  0.        , ..., 11.8       ,
         1.        ,  0.6       ],
       [11.        ,  4.        ,  0.        , ..., 46.09090909,
         2.        ,  0.9091    ]])

In [14]:
x_test_np

array([[ 2.        ,  1.        ,  0.        , ..., 23.5       ,
         4.        ,  2.        ],
       [ 2.        ,  1.        ,  0.        , ..., 25.        ,
         1.        ,  0.5       ],
       [ 9.        ,  1.        ,  0.        , ..., 67.66666667,
        30.        ,  5.2222    ],
       ...,
       [ 4.        ,  3.        ,  0.        , ..., 88.        ,
        16.        ,  5.5       ],
       [11.        ,  5.        ,  0.        , ..., 51.        ,
         2.        ,  0.9091    ],
       [ 6.        ,  5.        ,  0.        , ..., 28.33333333,
         3.        ,  1.        ]])

In [15]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [16]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [17]:
train_x

array([[[ 2.        ,  3.        ,  0.        , ..., 30.        ,
          3.        ,  1.5       ]],

       [[ 3.        ,  1.        ,  0.        , ...,  0.        ,
          1.        ,  1.        ]],

       [[ 7.        ,  5.        ,  0.        , ..., 11.        ,
          1.        ,  0.7143    ]],

       ...,

       [[11.        ,  4.        ,  0.        , ..., 32.81818182,
          2.        ,  0.9091    ]],

       [[ 5.        ,  5.        ,  0.        , ..., 11.8       ,
          1.        ,  0.6       ]],

       [[11.        ,  4.        ,  0.        , ..., 46.09090909,
          2.        ,  0.9091    ]]])

In [18]:
test_y

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       0, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0,

In [19]:
def define_model():
    # Designing and initializing the model.
    model = Sequential()
    model.add(LSTM(200, return_sequences=True))
    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(50, dropout = 0.2, return_sequences=True))
    model.add(LSTM(20, return_sequences=False))
    model.add(Dense(1, activation = 'relu'))
    model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0005) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )
    return model

In [20]:
y = list(test_y)

In [21]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(10):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/propv225/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(train_x, train_y, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(test_x, test_y)
    total_scores.append(score) 
    predictions = model.predict(test_x)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
15/15 [==============================] - 9s 42ms/step - loss: 0.2856 - mse: 0.2856 - mae: 0.1677 - root_mean_squared_error: 0.5344 - mean_squared_logarithmic_error: 0.0686
Epoch 2/100
15/15 [==============================] - 0s 21ms/step - loss: 0.2750 - mse: 0.2750 - mae: 0.2234 - root_mean_squared_error: 0.5244 - mean_squared_logarithmic_error: 0.0674
Epoch 3/100
15/15 [==============================] - 0s 22ms/step - loss: 0.2701 - mse: 0.2701 - mae: 0.2301 - root_mean_squared_error: 0.5197 - mean_squared_logarithmic_error: 0.0673
Epoch 4/100
15/15 [==============================] - 0s 22ms/step - loss: 0.2658 - mse: 0.2658 - mae: 0.2055 - root_mean_squared_error: 0.5155 - mean_squared_logarithmic_error: 0.0639
Epoch 5/100
15/15 [==============================] - 0s 24ms/step - loss: 0.2626 - mse: 0.2626 - mae: 0.2125 - root_mean_squared_error: 0.5125 - mean_squared_logarithmic_error: 0.0649
Epoch 6/100
15/15 [==============================] - 0s 22ms/step - lo

15/15 [==============================] - 0s 20ms/step - loss: 0.0927 - mse: 0.0927 - mae: 0.0865 - root_mean_squared_error: 0.3044 - mean_squared_logarithmic_error: 0.0308
Epoch 89/100
15/15 [==============================] - 0s 21ms/step - loss: 0.0897 - mse: 0.0897 - mae: 0.0856 - root_mean_squared_error: 0.2995 - mean_squared_logarithmic_error: 0.0309
Epoch 90/100
15/15 [==============================] - 0s 21ms/step - loss: 0.0898 - mse: 0.0898 - mae: 0.0823 - root_mean_squared_error: 0.2996 - mean_squared_logarithmic_error: 0.0288
Epoch 91/100
15/15 [==============================] - 0s 21ms/step - loss: 0.0947 - mse: 0.0947 - mae: 0.0864 - root_mean_squared_error: 0.3077 - mean_squared_logarithmic_error: 0.0320
Epoch 92/100
15/15 [==============================] - 0s 20ms/step - loss: 0.0871 - mse: 0.0871 - mae: 0.0841 - root_mean_squared_error: 0.2952 - mean_squared_logarithmic_error: 0.0295
Epoch 93/100
15/15 [==============================] - 0s 20ms/step - loss: 0.0875 - mse:

15/15 [==============================] - 0s 21ms/step - loss: 0.0945 - mse: 0.0945 - mae: 0.0946 - root_mean_squared_error: 0.3074 - mean_squared_logarithmic_error: 0.0312
Epoch 74/100
15/15 [==============================] - 0s 20ms/step - loss: 0.1031 - mse: 0.1031 - mae: 0.0944 - root_mean_squared_error: 0.3210 - mean_squared_logarithmic_error: 0.0328
Epoch 75/100
15/15 [==============================] - 0s 21ms/step - loss: 0.1275 - mse: 0.1275 - mae: 0.1040 - root_mean_squared_error: 0.3571 - mean_squared_logarithmic_error: 0.0391
Epoch 76/100
15/15 [==============================] - 0s 20ms/step - loss: 0.1081 - mse: 0.1081 - mae: 0.0988 - root_mean_squared_error: 0.3288 - mean_squared_logarithmic_error: 0.0339
Epoch 77/100
15/15 [==============================] - 0s 21ms/step - loss: 0.0993 - mse: 0.0993 - mae: 0.0973 - root_mean_squared_error: 0.3150 - mean_squared_logarithmic_error: 0.0329
Epoch 78/100
15/15 [==============================] - 0s 21ms/step - loss: 0.0914 - mse:

15/15 [==============================] - 0s 22ms/step - loss: 0.1046 - mse: 0.1046 - mae: 0.1019 - root_mean_squared_error: 0.3234 - mean_squared_logarithmic_error: 0.0349
Epoch 59/100
15/15 [==============================] - 0s 22ms/step - loss: 0.1116 - mse: 0.1116 - mae: 0.1050 - root_mean_squared_error: 0.3341 - mean_squared_logarithmic_error: 0.0363
Epoch 60/100
15/15 [==============================] - 0s 20ms/step - loss: 0.1013 - mse: 0.1013 - mae: 0.0980 - root_mean_squared_error: 0.3182 - mean_squared_logarithmic_error: 0.0326
Epoch 61/100
15/15 [==============================] - 0s 21ms/step - loss: 0.1077 - mse: 0.1077 - mae: 0.1063 - root_mean_squared_error: 0.3282 - mean_squared_logarithmic_error: 0.0356
Epoch 62/100
15/15 [==============================] - 0s 20ms/step - loss: 0.1108 - mse: 0.1108 - mae: 0.1069 - root_mean_squared_error: 0.3329 - mean_squared_logarithmic_error: 0.0361
Epoch 63/100
15/15 [==============================] - 0s 21ms/step - loss: 0.1099 - mse:

Epoch 1/100
15/15 [==============================] - 7s 48ms/step - loss: 0.2825 - mse: 0.2825 - mae: 0.1709 - root_mean_squared_error: 0.5315 - mean_squared_logarithmic_error: 0.0675
Epoch 2/100
15/15 [==============================] - 0s 21ms/step - loss: 0.2731 - mse: 0.2731 - mae: 0.2320 - root_mean_squared_error: 0.5226 - mean_squared_logarithmic_error: 0.0677
Epoch 3/100
15/15 [==============================] - 0s 21ms/step - loss: 0.2688 - mse: 0.2688 - mae: 0.2161 - root_mean_squared_error: 0.5185 - mean_squared_logarithmic_error: 0.0655
Epoch 4/100
15/15 [==============================] - 0s 21ms/step - loss: 0.2661 - mse: 0.2661 - mae: 0.2268 - root_mean_squared_error: 0.5158 - mean_squared_logarithmic_error: 0.0673
Epoch 5/100
15/15 [==============================] - 0s 21ms/step - loss: 0.2624 - mse: 0.2624 - mae: 0.2229 - root_mean_squared_error: 0.5123 - mean_squared_logarithmic_error: 0.0658
Epoch 6/100
15/15 [==============================] - 0s 21ms/step - loss: 0.2590

...

15/15 [==============================] - 0s 21ms/step - loss: 0.1048 - mse: 0.1048 - mae: 0.0985 - root_mean_squared_error: 0.3237 - mean_squared_logarithmic_error: 0.0344
Epoch 90/100
15/15 [==============================] - 0s 21ms/step - loss: 0.1054 - mse: 0.1054 - mae: 0.0954 - root_mean_squared_error: 0.3246 - mean_squared_logarithmic_error: 0.0339
Epoch 91/100
15/15 [==============================] - 0s 21ms/step - loss: 0.0934 - mse: 0.0934 - mae: 0.0938 - root_mean_squared_error: 0.3056 - mean_squared_logarithmic_error: 0.0325
Epoch 92/100
15/15 [==============================] - 0s 21ms/step - loss: 0.0935 - mse: 0.0935 - mae: 0.0923 - root_mean_squared_error: 0.3057 - mean_squared_logarithmic_error: 0.0321
Epoch 93/100
15/15 [==============================] - 0s 21ms/step - loss: 0.0931 - mse: 0.0931 - mae: 0.0901 - root_mean_squared_error: 0.3051 - mean_squared_logarithmic_error: 0.0310
Epoch 94/100
15/15 [==============================] - 0s 21ms/step - loss: 0.0966 - mse:

15/15 [==============================] - 0s 18ms/step - loss: 0.1142 - mse: 0.1142 - mae: 0.1038 - root_mean_squared_error: 0.3380 - mean_squared_logarithmic_error: 0.0333
Epoch 75/100
15/15 [==============================] - 0s 21ms/step - loss: 0.1063 - mse: 0.1063 - mae: 0.1023 - root_mean_squared_error: 0.3260 - mean_squared_logarithmic_error: 0.0332
Epoch 76/100
15/15 [==============================] - 0s 21ms/step - loss: 0.1007 - mse: 0.1007 - mae: 0.1030 - root_mean_squared_error: 0.3173 - mean_squared_logarithmic_error: 0.0322
Epoch 77/100
15/15 [==============================] - 0s 21ms/step - loss: 0.0951 - mse: 0.0951 - mae: 0.1017 - root_mean_squared_error: 0.3084 - mean_squared_logarithmic_error: 0.0316
Epoch 78/100
15/15 [==============================] - 0s 21ms/step - loss: 0.0983 - mse: 0.0983 - mae: 0.0970 - root_mean_squared_error: 0.3136 - mean_squared_logarithmic_error: 0.0325
Epoch 79/100
15/15 [==============================] - 0s 21ms/step - loss: 0.1000 - mse:

15/15 [==============================] - 0s 21ms/step - loss: 0.1281 - mse: 0.1281 - mae: 0.1148 - root_mean_squared_error: 0.3578 - mean_squared_logarithmic_error: 0.0377
Epoch 60/100
15/15 [==============================] - 0s 20ms/step - loss: 0.1248 - mse: 0.1248 - mae: 0.1212 - root_mean_squared_error: 0.3533 - mean_squared_logarithmic_error: 0.0372
Epoch 61/100
15/15 [==============================] - 0s 21ms/step - loss: 0.1244 - mse: 0.1244 - mae: 0.1011 - root_mean_squared_error: 0.3527 - mean_squared_logarithmic_error: 0.0357
Epoch 62/100
15/15 [==============================] - 0s 21ms/step - loss: 0.1272 - mse: 0.1272 - mae: 0.1221 - root_mean_squared_error: 0.3567 - mean_squared_logarithmic_error: 0.0380
Epoch 63/100
15/15 [==============================] - 0s 21ms/step - loss: 0.1187 - mse: 0.1187 - mae: 0.1046 - root_mean_squared_error: 0.3446 - mean_squared_logarithmic_error: 0.0364
Epoch 64/100
15/15 [==============================] - 0s 22ms/step - loss: 0.1310 - mse:

Epoch 1/100
15/15 [==============================] - 7s 47ms/step - loss: 0.2861 - mse: 0.2861 - mae: 0.1552 - root_mean_squared_error: 0.5349 - mean_squared_logarithmic_error: 0.0686
Epoch 2/100
15/15 [==============================] - 0s 21ms/step - loss: 0.2754 - mse: 0.2754 - mae: 0.2286 - root_mean_squared_error: 0.5248 - mean_squared_logarithmic_error: 0.0683
Epoch 3/100
15/15 [==============================] - 0s 20ms/step - loss: 0.2707 - mse: 0.2707 - mae: 0.2318 - root_mean_squared_error: 0.5203 - mean_squared_logarithmic_error: 0.0673
Epoch 4/100
15/15 [==============================] - 0s 21ms/step - loss: 0.2681 - mse: 0.2681 - mae: 0.1998 - root_mean_squared_error: 0.5178 - mean_squared_logarithmic_error: 0.0642
Epoch 5/100
15/15 [==============================] - 0s 20ms/step - loss: 0.2641 - mse: 0.2641 - mae: 0.2126 - root_mean_squared_error: 0.5139 - mean_squared_logarithmic_error: 0.0650
Epoch 6/100
15/15 [==============================] - 0s 21ms/step - loss: 0.2622

15/15 [==============================] - 0s 21ms/step - loss: 0.0937 - mse: 0.0937 - mae: 0.0941 - root_mean_squared_error: 0.3061 - mean_squared_logarithmic_error: 0.0290
Epoch 90/100
15/15 [==============================] - 0s 21ms/step - loss: 0.0945 - mse: 0.0945 - mae: 0.0950 - root_mean_squared_error: 0.3074 - mean_squared_logarithmic_error: 0.0304
Epoch 91/100
15/15 [==============================] - 0s 21ms/step - loss: 0.0904 - mse: 0.0904 - mae: 0.0928 - root_mean_squared_error: 0.3007 - mean_squared_logarithmic_error: 0.0291
Epoch 92/100
15/15 [==============================] - 0s 21ms/step - loss: 0.1073 - mse: 0.1073 - mae: 0.0958 - root_mean_squared_error: 0.3275 - mean_squared_logarithmic_error: 0.0323
Epoch 93/100
15/15 [==============================] - 0s 20ms/step - loss: 0.1048 - mse: 0.1048 - mae: 0.1040 - root_mean_squared_error: 0.3238 - mean_squared_logarithmic_error: 0.0343
Epoch 94/100
15/15 [==============================] - 0s 21ms/step - loss: 0.0962 - mse:

In [22]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/propv225/'

Reusing TensorBoard on port 6019 (pid 16875), started 0:32:51 ago. (Use '!kill 16875' to kill it.)

In [23]:
total_training_time

[46.61791968345642,
 43.89622378349304,
 45.26836681365967,
 44.13628387451172,
 43.857274770736694,
 44.06640696525574,
 44.27274179458618,
 45.916205644607544,
 43.841376066207886,
 44.39436411857605]

In [24]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 14.875572117169698


In [25]:
total_training_loss_history

[{'loss': [0.2855704724788666,
   0.2750204801559448,
   0.2701365351676941,
   0.2657527029514313,
   0.26261386275291443,
   0.2589646875858307,
   0.2548404335975647,
   0.2493322789669037,
   0.2446242868900299,
   0.23423269391059875,
   0.22712448239326477,
   0.21735315024852753,
   0.21822907030582428,
   0.2076531946659088,
   0.20110660791397095,
   0.193729966878891,
   0.18758904933929443,
   0.1827850341796875,
   0.1771220862865448,
   0.18894770741462708,
   0.17603561282157898,
   0.1788977086544037,
   0.1734415739774704,
   0.16039356589317322,
   0.1748933494091034,
   0.1855090707540512,
   0.17415393888950348,
   0.1597515195608139,
   0.16250093281269073,
   0.15572652220726013,
   0.14525406062602997,
   0.14061976969242096,
   0.14450061321258545,
   0.15266601741313934,
   0.13535435497760773,
   0.13152629137039185,
   0.14715775847434998,
   0.13581378757953644,
   0.12758785486221313,
   0.12296535074710846,
   0.12562359869480133,
   0.12097050994634628,
  

In [26]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(10):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 0.030921146273612976
avg mae of training: 0.029733922084172568


In [27]:
total_scores

[[0.3051650822162628,
  0.3051650822162628,
  0.19328591227531433,
  0.552417516708374,
  0.08997118473052979],
 [0.2693323493003845,
  0.2693323493003845,
  0.16964848339557648,
  0.5189723968505859,
  0.07751812040805817],
 [0.2040730118751526,
  0.2040730118751526,
  0.1550637185573578,
  0.45174440741539,
  0.06649694591760635],
 [0.27182894945144653,
  0.27182894945144653,
  0.19173157215118408,
  0.5213721990585327,
  0.08659462630748749],
 [0.2509151101112366,
  0.2509151101112366,
  0.1862645298242569,
  0.5009142756462097,
  0.08176766335964203],
 [0.264018177986145,
  0.264018177986145,
  0.1799943894147873,
  0.5138269662857056,
  0.08347466588020325],
 [0.24206414818763733,
  0.24206414818763733,
  0.17499861121177673,
  0.49200016260147095,
  0.07811487466096878],
 [0.21289679408073425,
  0.21289679408073425,
  0.15028603374958038,
  0.4614073932170868,
  0.06809680163860321],
 [0.21280114352703094,
  0.21280114352703094,
  0.1482124924659729,
  0.4613037407398224,
  0.066

In [1]:
a = [[0.3051650822162628,
  0.3051650822162628,
  0.19328591227531433,
  0.552417516708374,
  0.08997118473052979],
 [0.2693323493003845,
  0.2693323493003845,
  0.16964848339557648,
  0.5189723968505859,
  0.07751812040805817],
 [0.2040730118751526,
  0.2040730118751526,
  0.1550637185573578,
  0.45174440741539,
  0.06649694591760635],
 [0.27182894945144653,
  0.27182894945144653,
  0.19173157215118408,
  0.5213721990585327,
  0.08659462630748749],
 [0.2509151101112366,
  0.2509151101112366,
  0.1862645298242569,
  0.5009142756462097,
  0.08176766335964203],
 [0.264018177986145,
  0.264018177986145,
  0.1799943894147873,
  0.5138269662857056,
  0.08347466588020325],
 [0.24206414818763733,
  0.24206414818763733,
  0.17499861121177673,
  0.49200016260147095,
  0.07811487466096878],
 [0.21289679408073425,
  0.21289679408073425,
  0.15028603374958038,
  0.4614073932170868,
  0.06809680163860321],
 [0.21280114352703094,
  0.21280114352703094,
  0.1482124924659729,
  0.4613037407398224,
  0.06646142154932022],
 [0.20797522366046906,
  0.20797522366046906,
  0.1594230830669403,
  0.456043004989624,
  0.0675930455327034]]

In [2]:
test_mae = []
for i in range(10):
    test_mae.append(a[i][2])

In [3]:
sum(test_mae)/10

0.17089088261127472

In [31]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(10):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/30))
print("avg mae: " + str(sum_mae/30))
print("avg are: " + str(sum_are/30))

avg mse: 0.08136899967988333
avg mae: 0.056963627537091575
avg are: 0.028299570828676224


In [32]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 0.09471328929066658
median testing mse: 0.24648962914943695


In [33]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.0889684334397316
median testing mae: 0.1723235473036766


In [34]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.08356353640556335


In [35]:
test_ARE

[0.087016575,
 0.09392265,
 0.08747698,
 0.0801105,
 0.082412526,
 0.07872928,
 0.08379374,
 0.083333336,
 0.09023941,
 0.081952125]